# Imports

In [69]:
# Librairies
import numpy as np
import pandas as pd

In [70]:
# Fichiers
# url = 'sources/' # Jupyter (local)
url = 'https://raw.githubusercontent.com/gllmfrnr/OC/master/p4/sources/' # Github
customers =       pd.read_csv(url + 'customers.csv')
products =        pd.read_csv(url + 'products.csv')    
transactions =    pd.read_csv(url + 'transactions.csv')

In [71]:
# Styles
# Exemple : print(color.bold + 'Hello World !' + color.end)
class style:
   blue = '\033[94m'
   green = '\033[92m'
   red = '\033[91m'
   bold = '\033[1m'
   underline = '\033[4m'
   end = '\033[0m' 

In [72]:
# Fonction d'exploration des datasets
def exploration(df):
    print( # Sample
        style.red + 'SAMPLE ↓ \n' + style.end,
        df.sample(3), '\n'
        ) 
    print(style.red + 'INFO ↓ \n' + style.end)
    print(df.info(), '\n') # Colonnes, null-count, types d'objets
    for i in range(0, len(df.columns)):
        print(
            style.red + 'COLONNE',
            style.bold + df.columns[i], ': \n' + style.end, # Nom de colonne
             df.iloc[:, i].describe(), '\n', # .describe()
            sum(df.iloc[:, i].isna()), 'Nan \n', # Nbre de Nan 
            df.shape[0] - np.count_nonzero(df.iloc[:, i]), 'zéros \n', # Nbre de 0
            len(df.iloc[:, i].unique()), 'valeurs uniques : \n',
            df.iloc[:, i].unique(), '\n' # Nombre de valeurs uniques 
        )

In [73]:
# Fonction de recherche de clé primaire (une seule colonne)
def pk(df,col): # Ne pas oublier les guillemets en déclarant le paramètre col
    x = 0
    df = df.reset_index()  
    for i in range(0,len(df)):   
        
        searched = df.loc[i][col] # Valeur à rechercher
    
        s = df[df[col]==searched].drop_duplicates() # Rechercher cette valeur dans toute la df, et supprimer les duplicates pour vérifier combien de lignes on obtient
  
        if len(s) == 1: # Si on obtient une seule ligne, c'est a priori une clé primaire
            x += 1
       
    if x == len(df): # Si pour chaque ligne de la df, la requêtes a ajouté 1 à x, alors x = la longueur de la dataframe
        print('\'', col, '\' est bien une clé primaire.')
    else:
        print('\'', col, '\' n\'est pas une clé primaire.')

# Fonction de recherche sur toute la dataframe
def dataframekeys(df):
  for i in range(0, len(df.columns)):
      print(pk(df, df.columns[i]))

# Exploration

## Customers
- 3 colonnes
    - client_id → identifiant unique pour chaque client (aucun doublon)
    - sex → genre (2 valeurs possibles : f ou m)
    - birth → année de naissance
- Clé primaire : **'client_id'**

In [82]:
# dataframekeys(customers)

# ' client_id ' est bien une clé primaire.
# ' sex ' n'est pas une clé primaire.
# ' birth ' n'est pas une clé primaire.

In [74]:
exploration(customers)

SAMPLE ↓ 
      client_id sex  birth
6668    c_4209   f   1974
5798    c_7539   f   1987
8449    c_7557   m   1981 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB
None 

COLONNE client_id : 
 count       8623
unique      8623
top       c_7387
freq           1
Name: client_id, dtype: object 
 0 Nan 
 0 zéros 
 8623 valeurs uniques : 
 ['c_4410' 'c_7839' 'c_1699' ... 'c_5119' 'c_5643' 'c_84'] 

COLONNE sex : 
 count     8623
unique       2
top          f
freq      4491
Name: sex, dtype: object 
 0 Nan 
 0 zéros 
 2 valeurs uniques : 
 ['f' 'm'] 

COLONNE birth : 
 count    8623.000000
mean     1978.280877
std        16.919535
min      1929.000000
25%      1966.000000
50%      1

## Products
- 3 colonnes
  - id_prod : identifiant produit unique
  - price : prix (1455 valeurs différentes, pour 3287 produits-lignes)
  - categ : catégorie (3 valeurs différentes : 0, 1, 2)
- Clé primaire : **id_prod**

In [83]:
# dataframekeys(products)

# ' id_prod ' est bien une clé primaire.
# ' price ' n'est pas une clé primaire.
# ' categ ' n'est pas une clé primaire.

In [76]:
exploration(products)

SAMPLE ↓ 
      id_prod  price  categ
1822   0_834  15.77      0
1514  0_1067  12.99      0
1715   2_171  79.99      2 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB
None 

COLONNE id_prod : 
 count       3287
unique      3287
top       0_2247
freq           1
Name: id_prod, dtype: object 
 0 Nan 
 0 zéros 
 3287 valeurs uniques : 
 ['0_1421' '0_1368' '0_731' ... '0_802' '1_140' '0_1920'] 

COLONNE price : 
 count    3287.000000
mean       21.856641
std        29.847908
min        -1.000000
25%         6.990000
50%        13.060000
75%        22.990000
max       300.000000
Name: price, dtype: float64 
 0 Nan 
 0 zéros 
 1455 valeurs uniques : 
 [19.99  5.13 17.99 ... 1

## Transactions

In [78]:
exploration(transactions)

SAMPLE ↓ 
        id_prod                        date session_id client_id
204877    1_54  2021-09-10 07:19:55.645567    s_87662    c_5033
84141   0_1102  2021-10-07 20:09:09.366399   s_101465    c_1996
296702  0_1033  2021-04-18 10:35:16.368773    s_22247     c_236 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB
None 

COLONNE id_prod : 
 count     337016
unique      3266
top        1_369
freq        1081
Name: id_prod, dtype: object 
 0 Nan 
 0 zéros 
 3266 valeurs uniques : 
 ['0_1483' '2_226' '1_374' ... '0_833' '0_1284' '0_1116'] 

COLONNE date : 
 count                              337016
unique                             336

In [79]:
dataframekeys(transactions)

KeyboardInterrupt: ignored

# Nettoyage